In [ ]:
import pandas, re, cleantext, nltk
import pandas as pd
from itertools import chain
from nltk.corpus import stopwords

In [ ]:
def clean(csv):
  df = pandas.read_csv(csv)
  for i in range(len(df["content"])):
    df.loc[i, "content"] = cleantext.clean(df.loc[i, "content"], lower=True, no_line_breaks=True, no_numbers=True, no_emails=True, no_urls=True, replace_with_number="<NUM>")
  return df

clean("news_sample.csv").to_csv("cleaned_news_sample.csv")

In [ ]:
def tokenize(dataFrame):
    df = dataFrame

    # Create empty list to gather lists of tokens
    # and loop through column to add words
    allWords = []

    for cell in df['content']:
        # words = cell.split()
        # allWords.append(list(words))
        tokens = nltk.word_tokenize(cell)
        allWords.append(tokens)
    
    # Create flat list from list of lists
    flat_list_raw = [word for inner_list in allWords for word in inner_list]
    flat_list_final = [word for item in flat_list_raw for word in re.split(r'[().,\'"-]', item) if word.strip()]
    # flat_list_final = [word for item in flat_list_raw for word in item.split('.')]
           
    return flat_list_final


In [ ]:
def stem(input):
  stemmer = nltk.SnowballStemmer(language="english", ignore_stopwords=True)
  for token in input:
    stemmer.stem(token)
  return input

In [ ]:
def remove_stopwords(input):
  for token in input:
    stop_words = stopwords.words('english')
    filteredInput = []
    
    for token in input:
      if token not in stop_words:
        filteredInput.append(token)
        
    return filteredInput

In [ ]:
def run():
  data = "news_sample.csv"
  cleaned = clean(data)
  tokenizedData = tokenize(cleaned)
  stopwordsRemoved = remove_stopwords(tokenizedData)
  
  for x in stopwordsRemoved:
    print(x)

run()